In [2]:
%load_ext autoreload
%autoreload 2

import torch 
from src.utils.iwo import complete_orthonormal_basis, get_basis

L = 10
W_list = [torch.rand(l - 1, l).to(torch.float64) for l in reversed(range(2, L + 1))]
B = get_basis(W_list)

# Test orthogonality
eye = torch.eye(B.size(0), dtype=B.dtype, device=B.device)
assert torch.allclose(B.t() @ B, eye, atol=1e-08)

# Test that the basis vectors are indeed inside the null space of the next smaller matrix
B_flipped = torch.flip(B, dims=[1])  # Re-order from least important to most important
for i in range(len(W_list) - 1):
    t = B_flipped[:, i : i + 1]
    for j in range(i + 1):
        t = W_list[j] @ t
    # Assert if the the projection is indeed inside the null-space.
    assert all(torch.le(t, 1e-6))